## Librerie

In [1]:
from scipy.optimize import minimize
import numpy as np
import pandas as pd

## Parametri problema

In [2]:
# Consumi famiglia 1, famiglia 2, ...
consumi = np.array([1500, 5700, 3400, 300])

# Produzioni ricetta1, ricetta2, ...
produzioni = np.array([3000, 6300, 1200])

# Ricette
#           | Ricetta1 | Ricetta2 | ...
# --------------------------------------
# Famiglia1 |          |          |
# Famiglia2 |          |          |
# ...
ricette = np.array([
    0.25, 0.2, 0.3,
    0.43, 0.5, 0.35,
    0.3, 0.27, 0.35,
    0.02, 0.03,  0
])

# Composizioni ricette per famiglia
#           | Materiale1 | Materiale2 | ...
# --------------------------------------
# Famiglia1 |            |            |
# Famiglia2 |            |            |
# ...
composizioni_famiglia = np.array([
    0.58, 0.42, 0,
    1, 0, 0,
    0, 1, 0,
    0, 0, 1
])

## Calcolo resa globale

In [3]:
tot_consumi = np.sum(consumi)
tot_produzioni = np.sum(produzioni)
resa_globale = tot_consumi / tot_produzioni
f'{resa_globale=}'

'resa_globale=np.float64(1.0380952380952382)'

## Funzioni di calcolo

In [4]:
# Calcola matrice consumi moltiplicando matrice ricetta in input per produzioni
def calc_mat_consumi(ricetta):
    return ricetta.reshape(-1, len(produzioni)) * produzioni

In [5]:
# Calcola vettore consumi complessivi partendo da produzioni iniziali e matrice consumi
def calc_tot_consumi(matrice_consumi):
    return np.sum(matrice_consumi, axis=1)

In [6]:
# Calcolo errore su totali consumi
def calc_err_totali(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_consumi = calc_tot_consumi(matrice_consumi)
    tot_err = np.sum(np.square(tot_consumi-consumi))
    return tot_err

In [7]:
# Calcola rese per famiglia 
# (consumi per famiglia / produzione)
def calc_tot_resa(matrice_consumi):
    return np.sum(matrice_consumi, axis=0)/produzioni

In [8]:
# Calcolo errore su percentuali prod. effettive rispetto a resa totale
def calc_error_resa(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_resa = calc_tot_resa(matrice_consumi)
    return np.sum(np.square(tot_resa - resa_globale))

## Analisi composizione

In [9]:
# Matrice consumi usando ricetta iniziale
cons_fam = calc_mat_consumi(ricette)
pd.DataFrame(cons_fam)

,0,1,2
0,750.0,1260.0,360.0
1,1290.0,3150.0,420.0
2,900.0,1701.0,420.0
3,60.0,189.0,0.0


In [10]:
# Reshape della matrice composizioni (famiglia vs materiale)
compos = composizioni_famiglia.reshape(len(consumi), -1)
pd.DataFrame(compos, columns=['Mat1', 'Mat2', 'Mat3'])

,Mat1,Mat2,Mat3
0,0.58,0.42,0.0
1,1.00,0.00,0.0
2,0.00,1.00,0.0
3,0.00,0.00,1.0


In [11]:
# Devo calcolare quanto materiale per ciascuna ricetta

# Proviamo per prima colonna matrice consumi (quindi prima ricetta)
# Moltiplico la colonna consumi di quella ricetta per le singole colonne della matrice composizione

cons_ricetta0 = np.vstack(cons_fam[:,0])
print(cons_ricetta0)
print('')
cons_materiali_ricetta0 = cons_ricetta0 * compos
print(cons_materiali_ricetta0)
print('')
tot_cons_ricetta0 = np.sum(cons_materiali_ricetta0, axis=0)
print(tot_cons_ricetta0)
print('')
print('Composizione ricetta 0')
tot_perc_ricetta0 = tot_cons_ricetta0 / np.sum(tot_cons_ricetta0)
print(tot_perc_ricetta0)

[[ 750.]
 [1290.]
 [ 900.]
 [  60.]]

[[ 435.  315.    0.]
 [1290.    0.    0.]
 [   0.  900.    0.]
 [   0.    0.   60.]]

[1725. 1215.   60.]

Composizione ricetta 0
[0.575 0.405 0.02 ]


In [12]:
def perc_mat(ricetta, id_ric):
    cons_fam = calc_mat_consumi(ricetta)
    compos = composizioni_famiglia.reshape(len(consumi), -1)
    cons_ricetta0 = np.vstack(cons_fam[:,id_ric])
    cons_materiali_ricetta0 = cons_ricetta0 * compos
    tot_cons_ricetta0 = np.sum(cons_materiali_ricetta0, axis=0)
    tot_perc_ricetta0 = tot_cons_ricetta0 / np.sum(tot_cons_ricetta0)
    return tot_perc_ricetta0

In [13]:
def err_perc_mat(ricetta, id_ric, id_mat, expected_val, expected_error):
    return expected_error - np.abs(perc_mat(ricetta, id_ric=id_ric)[id_mat] - expected_val)

## Ottimizzazione

In [14]:
costraints = (
    {'type': 'eq', 'fun': calc_err_totali},
    {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 0, 0.58, 0.01)},
    {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 1, 0.396, 0.02)},
    {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 2, 0.024, 0.001)},

    # {'type': 'ineq', 'fun': err_perc_mat, 'args': (1, 0, 0.625, 0.005)},

    {'type': 'ineq', 'fun': err_perc_mat, 'args': (2, 0, 0.62, 0.01)},
)

bounds = list(( (0, None) for x in range(len(produzioni)*len(consumi)) ))
# bounds[4] = (0, 0.5) #DEBUG

res = minimize(
    calc_error_resa, 
    ricette, 
    method='SLSQP',
    constraints=costraints,
    bounds=bounds,
    options={'disp': True, 'maxiter':100}
)
print(res)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6294923616249576e-16
            Iterations: 26
            Function evaluations: 342
            Gradient evaluations: 26
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1.6294923616249576e-16
       x: [ 2.022e-01  8.502e-02  2.981e-01  4.856e-01  5.824e-01
            4.786e-01  3.256e-01  3.362e-01  2.542e-01  2.471e-02
            3.448e-02  7.222e-03]
     nit: 26
     jac: [-2.281e-11 -2.354e-10 -2.693e-10 -2.281e-11 -2.354e-10
           -2.693e-10 -2.281e-11 -2.354e-10 -2.693e-10 -2.281e-11
           -2.354e-10 -2.693e-10]
    nfev: 342
    njev: 26


## Verifiche

In [15]:
pd.options.display.float_format = '{:.2f}'.format

In [16]:
print("Percentuali aggiustate (in %)")
pd.DataFrame(
    res.x.reshape(len(consumi), len(produzioni))*100
)

Percentuali aggiustate (in %)


,0,1,2
0,20.22,8.50,29.81
1,48.56,58.24,47.86
2,32.56,33.62,25.42
3,2.47,3.45,0.72


In [17]:
print("Matrice consumi")
pd.DataFrame(
    calc_mat_consumi(res.x)
)

Matrice consumi


,0,1,2
0,606.69,535.64,357.67
1,1456.72,3668.99,574.29
2,976.74,2118.17,305.09
3,74.13,217.20,8.67


In [18]:
print('Verifica totale consumi')
pd.DataFrame(
    [calc_tot_consumi(calc_mat_consumi(res.x))]
)

Verifica totale consumi


,0,1,2,3
0,1500.00,5700.00,3400.00,300.00


In [19]:
print('Verifica errore consumi')
pd.DataFrame([calc_tot_consumi(calc_mat_consumi(res.x)) - consumi])

Verifica errore consumi


,0,1,2,3
0,-0.00,0.00,-0.00,0.00


In [20]:
print(f'Verifica rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x))]
)

Verifica rese (resa globale: 1.04)


,0,1,2
0,1.04,1.04,1.04


In [21]:
print(f'Verifica errore rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x)) - resa_globale]
)

Verifica errore rese (resa globale: 1.04)


,0,1,2
0,-0.00,-0.00,-0.00


In [22]:
print('Percentuali materiali ricetta 0')
pd.DataFrame([perc_mat(res.x, 0) * 100])

Percentuali materiali ricetta 0


,0,1,2
0,58.07,39.55,2.38


In [23]:
print('Percentuali materiali ricetta 1')
pd.DataFrame([perc_mat(res.x, 1) * 100])

Percentuali materiali ricetta 1


,0,1,2
0,60.85,35.83,3.32


In [24]:
print('Percentuali materiali ricetta 2')
pd.DataFrame([perc_mat(res.x, 2) * 100])

Percentuali materiali ricetta 2


,0,1,2
0,62.75,36.55,0.70


# ?

In [25]:
matrice_consumi = calc_mat_consumi(res.x)
# tot_resa = calc_tot_resa(matrice_consumi)
tot_resa = np.sum(matrice_consumi, axis=0)/produzioni
tot_resa
# np.sum(np.square(tot_resa - resa_globale))
print(np.sum(matrice_consumi, axis=0))
print(produzioni)

[3114.28569256 6539.99995236 1245.71427695]
[3000 6300 1200]
